# Setup and Load dataset



In [ ]:
!pip install rank_bm25 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
!gdown --id 1EvuYBN9lIK8llpbUciP5jIOYgh0dpFHB

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1EvuYBN9lIK8llpbUciP5jIOYgh0dpFHB
To: /content/bengali.zip
100% 13.5M/13.5M [00:00<00:00, 127MB/s]


In [ ]:
!unzip -o "bengali.zip"  -d  "/content"

Archive:  bengali.zip
  inflating: /content/train.csv      
  inflating: /content/valid.csv      


## Data Loading

In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

#### Name of the file which contain all the item properties

In [ ]:
file="train.csv"

###### Run below cell

In [ ]:
content_df  = pd.read_csv( file)

In [ ]:
content_df.head(10)

,title,article,label
0,এই অভিনেতার ‘প্রস্তাব’ মানলে ভারতীয় সিনেমার খো...,স্রেফ দু’টি টুইটেই সোশ্যাল মিডিয়ায় তোলপাড় ফে...,entertainment
1,"সচিন সাংসদ হয়েছেন, তাই চিন্তা কম জেটলির। কেন?",একবার রেল বাজেট। একবার সাধারণ বাজেট। খবরের শি...,national
2,ক্রিকেটার থেকে গাইড! জাতীয় দলের ক্রিকেটারের অব...,কিছুদিন আগেই রাজ্যের গাইড বনে গিয়েছিলেন চেতেশ...,sports
3,"দুষ্কৃতীদের নজরে এটিএম, হাল হকিকত্ ঘুরে দেখলেন...",কলকাতায় দুষ্কৃতীদের নজরে এখন বিভিন্ন ব্যাঙ্কের...,kolkata
4,"সৌজন্যের বালাই নেই, দিনভর আকচা আকচি কং-বিজেপি’র",সারা দিন আকচা আকচিতেই কাটিয়ে দিল দেশের দুই বৃহ...,national
5,নোট বদলের ধাক্কায় কেমন হল বিয়ে? জানুন,বিরিয়নিটা আর হোল না। ইচ্ছে ছিল বিয়েতে বিরায়ান...,kolkata
6,সিপিএমের মুখিয়া,এস কে এম কি আছেন?\nএ খোঁজ যে রাজ্য সিপিএমের ম...,state
7,গড়িয়ায় দফায় দফায় গণধর্ষণের শিকার অষ্টম শ্রেণির...,মহানগরের বুকে এবার দফায় দফায় গণধর্ষণের শিকার...,kolkata
8,বামেদের ধর্মঘটকে সমর্থন! নবান্ন অভিযানে বামেরা...,"আগামী বুধবার, ২ সেপ্টেম্বর বামেদের ডাকা বনধকে...",kolkata
9,আকাশবাড়ির হাতছানি,"পেরুতে গিয়ে যদি হোটেলের ঘরে বসে সময় কাটান, তা...",international


## Details about dataset

In [ ]:
itemid="title"

In [ ]:
features=['title','article']

In [ ]:
allcols=[itemid]
for i in features:
  allcols.append(i)

# Setup

In [ ]:
content_df['NewTag']=""

for i in features:
  content_df[i] = content_df[i].fillna(' ')
for i in features:
  content_df['NewTag']+=(' '+content_df[i])
content_df['NewTag']=content_df['NewTag'].astype(str)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [ ]:
def clean_text(text):
    
    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub( re.compile("\'s"), ' ', text)
    text = re.sub(re.compile("\\r\\n"), ' ', text)
    text = re.sub(re.compile(r"[^\w\s]"), ' ', text)
    return text

In [ ]:
stopwords=set(stopwords.words('bengali'))

In [ ]:
def stopwprds_removal_gensim_custom(str):
  
    text_tokens = word_tokenize(str)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords]
    str_t = " ".join(tokens_without_sw)

    return str_t

In [ ]:

content_df['clean'] = content_df['NewTag'].apply(stopwprds_removal_gensim_custom) 


In [ ]:
tokenized_corpus = [doc.split(" ") for doc in content_df['clean'].values]

# Gensim BM25

In [ ]:
from gensim import corpora
from gensim.summarization import bm25

text_token = [doc.split() for doc in content_df['clean'].values] # you can do preprocessing as removing stopwords
# dictionary = corpora.Dictionary(texts)
# corpus = [dictionary.doc2bow(text) for text in texts]
bm25_obj = bm25.BM25(text_token)


In [ ]:
search_query="ক্রিকেটার থেকে গাইড"
query_doc = search_query.split()


In [ ]:
scores = bm25_obj.get_scores(query_doc,text_token)
best_docs = sorted(range(len(scores)), key=lambda i: scores[i])[-10:]
best_docs

[4124, 2200, 8297, 5208, 9002, 9822, 8128, 5146, 9625, 2]

In [ ]:
itemids=[]
for i in best_docs:
  itemids.append(content_df.iloc[i][itemid])

In [ ]:
content_df[content_df[itemid].isin(itemids)]

,title,article,label,NewTag,clean
2,ক্রিকেটার থেকে গাইড! জাতীয় দলের ক্রিকেটারের অব...,কিছুদিন আগেই রাজ্যের গাইড বনে গিয়েছিলেন চেতেশ...,sports,ক্রিকেটার থেকে গাইড! জাতীয় দলের ক্রিকেটারের অ...,ক্রিকেটার গাইড ! জাতীয় দলের ক্রিকেটারের অবাক ভ...
2200,বিরাট-ধোনি নন! সুন্দরী সোনাক্ষীর হৃদয়ে কোন ক্র...,আইপিএল মানেই বিনোদন আর ধুমধাড়াক্কা ক্রিকেটের...,sports,বিরাট-ধোনি নন! সুন্দরী সোনাক্ষীর হৃদয়ে কোন ক্...,বিরাট-ধোনি নন ! সুন্দরী সোনাক্ষীর হৃদয়ে ক্রিকে...
4124,"মাঠেই মৃত্যু প্রাক্তন ভারতীয় ক্রিকেটারের, অকাল...",প্রাক্তন রঞ্জি ক্রিকেটার এবং কোচ অমৃক সিংহ না...,sports,"মাঠেই মৃত্যু প্রাক্তন ভারতীয় ক্রিকেটারের, অকা...","মাঠেই মৃত্যু প্রাক্তন ভারতীয় ক্রিকেটারের , অকা..."
5146,"তোলাবাজদের এবার দলেও জায়গা নেই, হুঁশিয়ারি মমতা...",সিন্ডিকেট তোলাবাজির দাপট কোনও ভাবেই বরদাস্ত ...,kolkata,"তোলাবাজদের এবার দলেও জায়গা নেই, হুঁশিয়ারি মমত...","তোলাবাজদের দলেও জায়গা , হুঁশিয়ারি মমতা বন্দ্যো..."
5208,আইসিসি-র সেরার তালিকায় কোনও ভারতীয় নেই,আইসিসি-র সেরার তালিকায় জায়গা পেলেন না কোনও ভা...,sports,আইসিসি-র সেরার তালিকায় কোনও ভারতীয় নেই আইসিস...,আইসিসি-র সেরার তালিকায় ভারতীয় আইসিসি-র সেরার ত...
8128,হুগলীতে তৃণমূলের 'মাথাব্যাথা' গোষ্ঠী দ্বন্দ্ব...,কোনও ভাবে বরদাস্ত করা হবে না গোষ্ঠী দ্বন্দ্...,kolkata,হুগলীতে তৃণমূলের 'মাথাব্যাথা' গোষ্ঠী দ্বন্দ্...,হুগলীতে তৃণমূলের ' মাথাব্যাথা ' গোষ্ঠী দ্বন্দ...
8297,কোহলির হাতে বিশ্বকাপ দেখতে বাজি দুই তারকা ক্রি...,২০১৯ বিশ্বকাপের দিকে তাকিয়ে রয়েছেন ভারতের নির...,sports,কোহলির হাতে বিশ্বকাপ দেখতে বাজি দুই তারকা ক্র...,"কোহলির হাতে বিশ্বকাপ বাজি তারকা ক্রিকেটার , ভা..."
9002,সা রে গা মা পা মঞ্চে নজিরবিহীন সিদ্ধান্ত! জিত ...,এবছরের ‘সা রে গা মা পা লিল চ্যাম্পস’ প্রতিযোগ...,entertainment,সা রে গা মা পা মঞ্চে নজিরবিহীন সিদ্ধান্ত! জিত...,সা রে গা মা পা মঞ্চে নজিরবিহীন সিদ্ধান্ত ! জিত...
9625,আজ হাইকোর্টে টেট মামলার রায় ঘোষণা,আজ হাইকোর্টে টেট মামলার রায় ঘোষণা। দুহাজার বা...,kolkata,আজ হাইকোর্টে টেট মামলার রায় ঘোষণা আজ হাইকোর্...,হাইকোর্টে টেট মামলার রায় ঘোষণা হাইকোর্টে টেট ম...
9822,তালিবানের আতঙ্ক দূর হয়ে ক্রিকেট এখন কবুল,ইডেন গার্ডেন্স দেখার পর অভিভূত হয়ে জানতে চান—...,sports,তালিবানের আতঙ্ক দূর হয়ে ক্রিকেট এখন কবুল ইডে...,তালিবানের আতঙ্ক দূর হয়ে ক্রিকেট কবুল ইডেন গার্...


# Rank BM25Okapi

In [ ]:
from rank_bm25 import *

In [ ]:
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
search_query="ক্রিকেটার থেকে গাইড"
tokenized_query = search_query.split(" ")
doc_scores = bm25.get_scores(tokenized_query)
docs = bm25.get_top_n(query=tokenized_query, documents=content_df['clean'].values, n=10)
df_search = content_df[content_df["clean"].isin(docs)]
df_search.head()


,title,article,label,NewTag,clean
2,ক্রিকেটার থেকে গাইড! জাতীয় দলের ক্রিকেটারের অব...,কিছুদিন আগেই রাজ্যের গাইড বনে গিয়েছিলেন চেতেশ...,sports,ক্রিকেটার থেকে গাইড! জাতীয় দলের ক্রিকেটারের অ...,ক্রিকেটার গাইড ! জাতীয় দলের ক্রিকেটারের অবাক ভ...
2200,বিরাট-ধোনি নন! সুন্দরী সোনাক্ষীর হৃদয়ে কোন ক্র...,আইপিএল মানেই বিনোদন আর ধুমধাড়াক্কা ক্রিকেটের...,sports,বিরাট-ধোনি নন! সুন্দরী সোনাক্ষীর হৃদয়ে কোন ক্...,বিরাট-ধোনি নন ! সুন্দরী সোনাক্ষীর হৃদয়ে ক্রিকে...
4124,"মাঠেই মৃত্যু প্রাক্তন ভারতীয় ক্রিকেটারের, অকাল...",প্রাক্তন রঞ্জি ক্রিকেটার এবং কোচ অমৃক সিংহ না...,sports,"মাঠেই মৃত্যু প্রাক্তন ভারতীয় ক্রিকেটারের, অকা...","মাঠেই মৃত্যু প্রাক্তন ভারতীয় ক্রিকেটারের , অকা..."
5146,"তোলাবাজদের এবার দলেও জায়গা নেই, হুঁশিয়ারি মমতা...",সিন্ডিকেট তোলাবাজির দাপট কোনও ভাবেই বরদাস্ত ...,kolkata,"তোলাবাজদের এবার দলেও জায়গা নেই, হুঁশিয়ারি মমত...","তোলাবাজদের দলেও জায়গা , হুঁশিয়ারি মমতা বন্দ্যো..."
5208,আইসিসি-র সেরার তালিকায় কোনও ভারতীয় নেই,আইসিসি-র সেরার তালিকায় জায়গা পেলেন না কোনও ভা...,sports,আইসিসি-র সেরার তালিকায় কোনও ভারতীয় নেই আইসিস...,আইসিসি-র সেরার তালিকায় ভারতীয় আইসিসি-র সেরার ত...
